# Single Agent Snek
In this notebook, we want to analyze the performances, critical features and results for the single agent environment.

## Draft roadmap
- Loading and rendering of the environment on notebook (save on GIF)
- Optimal solution performance and rendering
- DQN experiments
    - Network size
    - Learnt model visual reconstruction
- ...

## Visualizing the environment

In [1]:
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML, Image
from IPython import display
import gym, sneks
import numpy as np
import sys

rc('animation', html='html5')

In [2]:
env = gym.make('snek-rgb-16-v1')

In [3]:
def play_one_episode(policy):
    obs = env.reset()
    observations, done, steps, reward = [obs], False, 0, 0
    while not done:
        obs, r, done, _ = env.step(policy(obs))
        reward += r
        steps += 1
        observations.append(obs)
    return observations, steps, reward

In [4]:
observations, steps, reward = play_one_episode(lambda x: env.action_space.sample())
print("Episode return:", reward)
print("Episode length:", steps)

Episode return: -1
Episode length: 14


In [5]:
def saveualize(observations, filepath):
    fig, ax = plt.subplots(1, figsize=(6, 6))
    fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)
    img = ax.imshow(observations[0])

    def animate(i):
        img.set_data(observations[i])
        return img

    plt.axis('off')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    anim = animation.FuncAnimation(fig, animate, frames=len(observations), interval=1000);
    anim.save(filepath, writer='imagemagick', fps=15)
    return Image(url=filepath, format='gif')

display.display(saveualize(observations, 'gifs/test_env.gif'))

## Simple policy

In [6]:
sys.path.append('..')
from basic.benchmark import BasicPolicy

In [12]:
pi = BasicPolicy()
observations, steps, reward = play_one_episode(lambda x: pi.act(x))
print("Episode return:", reward)
print("Episode length:", steps)

Episode return: 21
Episode length: 313


In [13]:
display.display(saveualize(observations, 'gifs/basic_policy.gif'))